### A. Goldman, N. Puchkin, V. Shcherbakova, and U. Vinogradova

### Numerical experiments on artificial data sets, described in the paper
### "A Contrastive Approach to Online Change Point Detection" (arXiv:2206.10143)

In [1]:
import numpy as np
from numpy.random import randn
from numpy.random import laplace
import pandas as pd
import matplotlib.pyplot as plt

# Import the algorithms for comparison
from algorithms.contrastive_change_point import compute_test_stat_linear
from algorithms.contrastive_change_point import compute_test_stat_nn
from algorithms.fast_contrastive_change_point import compute_test_stat_ftal
from algorithms.kliep import compute_test_stat_kliep
from algorithms.m_statistic import compute_test_stat_mmd
from algorithms.cusum import compute_cusum

# Set the thresholds as recorded in the files
# in the 'thresholds' folder
from thresholds.consts import *

%matplotlib inline

### Mean shift detection

In [2]:
np.random.seed(1)

# Number of observations
n = 150
# True change point
tau = 75

# Shift size
mu = 0.2

# Standard deviation of the observations
sigma = 0.1

p = 2
S_poly_list = []
S_nn_list = []

for item in range(10):
    
    print('Iteration', item)
    
    # Generate a Gaussian sequence of observations
    X = sigma * randn(n)
    X[tau:] += mu

    S_poly, _ = compute_test_stat_linear(X, p)
    S_nn, _ = compute_test_stat_nn(X, n_epochs=50)
    
    S_poly_list.append(S_poly)
    S_nn_list.append(S_nn)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [3]:
t_lin = []
t_nn = []

fa_lin = 0
fa_nn = 0

for i in range(10):

    S_lin = S_poly_list[i]
    S_nn = S_nn_list[i]
    imin_lin, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_lin, S_lin <= threshold_p_2_poly))
    imin_nn, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_nn, S_nn <= threshold_nn))
    
    if imin_lin - tau <= 0:
        fa_lin += 1
    else:
        t_lin.append(imin_lin - tau)
        
    if imin_nn - tau <= 0:
        fa_nn += 1
    else:
        t_nn.append(imin_nn - tau)
        
print('Neural networks. False alarms:', fa_nn)
print('Polynomials. False alarms:', fa_lin)

Neural networks. False alarms: 0
Polynomials. False alarms: 0


In [4]:
t_lin_np = np.array(t_lin)
DD_poly = np.round(t_lin_np.mean(), 1)
std_DD_poly = np.round(t_lin_np.std(), 1)
print('polynomial basis', DD_poly, '±', std_DD_poly)

t_nn_np = np.array(t_nn)
DD_nn = np.round(t_nn_np.mean(), 1)
std_DD_nn = np.round(t_nn_np.std(), 1)
print('neural network', DD_nn, '±', std_DD_nn)

polynomial basis 6.7 ± 2.0
neural network 8.9 ± 1.2


In [5]:
np.random.seed(1)

# Number of observations
n = 150
# True change point
tau = 75

# Shift size
mu = 0.2

# Standard deviation of the observations
sigma = 0.1

p = 2
S_hermite_list = []
S_legendre_list = []

for item in range(10):
    
    print('Iteration', item)
    
    # Generate a Gaussian sequence of observations
    X = sigma * randn(n)
    X[tau:] += mu

    S_hermite, _ = compute_test_stat_ftal(X, p, beta=0.01, design='hermite')
    S_legendre, _ = compute_test_stat_ftal(X, p, beta=0.01, design='legendre')
    
    S_hermite_list.append(S_hermite)
    S_legendre_list.append(S_legendre)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [6]:
t_hermite = []
t_legendre = []

fa_hermite = 0
fa_legendre = 0

for i in range(10):

    S_hermite = S_hermite_list[i]
    S_legendre = S_legendre_list[i]
    imin_hermite, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_hermite,\
                                                                   S_hermite <= threshold_hermite_p_2_001))
    imin_legendre, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_legendre,\
                                                                    S_legendre <= threshold_legendre_p_2_001))
    
    if imin_hermite - tau <= 0:
        fa_hermite += 1
    else:
        t_hermite.append(imin_hermite - tau)
        
    if imin_legendre - tau <= 0:
        fa_legendre += 1
    else:
        t_legendre.append(imin_legendre - tau)
        
print('Hermite polynomials. False alarms:', fa_hermite)
print('Legendre polynomials. False alarms:', fa_legendre)

Hermite polynomials. False alarms: 0
Legendre polynomials. False alarms: 0


In [7]:
t_hermite_np = np.array(t_hermite)
DD_hermite = np.round(t_hermite_np.mean(), 1)
std_DD_hermite = np.round(t_hermite_np.std(), 1)
print('FTAL Hermite', DD_hermite, '±', std_DD_hermite)

t_legendre_np = np.array(t_legendre)
DD_legendre = np.round(t_legendre_np.mean(), 1)
std_DD_legendre = np.round(t_legendre_np.std(), 1)
print('FTAL Legendre', DD_legendre, '±', std_DD_legendre)

FTAL Hermite 5.5 ± 1.7
FTAL Legendre 5.5 ± 1.7


In [8]:
np.random.seed(1)

# Number of observations
n = 150
# True change point
tau = 75

# Shift size
mu = 0.2

# Standard deviation of the observations
sigma = 0.1

S_kliep_list = []
S_mmd_list = []
S_cusum_list = []

for item in range(10):
    
    print('Iteration', item)
    
    # Generate a Gaussian sequence of observations
    X = sigma * randn(n)
    X[tau:] += mu

    S_kliep, _ = compute_test_stat_kliep(X, window_size=20, sigma=0.2)
    S_mmd, _ = compute_test_stat_mmd(X, window_size=20, sigma=0.5)
    S_cusum = compute_cusum(X)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    S_cusum_list.append(S_cusum)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [9]:
t_kliep = []
t_mmd = []
t_cusum = []

fa_kliep = 0
fa_mmd = 0
fa_cusum = 0

for i in range(10):

    S_kliep = S_kliep_list[i]
    S_mmd = S_mmd_list[i]
    S_cusum = S_cusum_list[i]
    
    if np.sum(S_kliep > threshold_kliep_20) > 0:
        imin_kliep, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_kliep, S_kliep <= threshold_kliep_20))
    else:
        imin_kliep = n
    
    if np.sum(S_mmd > threshold_mmd_20_50) > 0:
        imin_mmd, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_mmd, S_mmd <= threshold_mmd_20_50))
    else:
        imin_mmd = n
        
    if np.sum(S_cusum > threshold_cusum) > 0:
        imin_cusum, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_cusum, S_cusum <= threshold_cusum))
    else:
        imin_cusum = n
    
    
    if imin_kliep - tau <= 0:
        fa_kliep += 1
    else:
        t_kliep.append(imin_kliep - tau)
        
    if imin_mmd - tau <= 0:
        fa_mmd += 1
    else:
        t_mmd.append(imin_mmd - tau)
        
    if imin_cusum - tau <= 0:
        fa_cusum += 1
    else:
        t_cusum.append(imin_cusum - tau)
        
print('KLIEP. False alarms:', fa_kliep)
print('M-statistic. False alarms:', fa_mmd)
print('CUSUM. False alarms:', fa_cusum)

KLIEP. False alarms: 0
M-statistic. False alarms: 0
CUSUM. False alarms: 0


In [10]:
t_kliep_np = np.array(t_kliep)
DD_kliep = np.round(t_kliep_np.mean(), 1)
std_DD_kliep = np.round(t_kliep_np.std(), 1)
print('KLIEP', DD_kliep, '±', std_DD_kliep)

t_mmd_np = np.array(t_mmd)
DD_mmd = np.round(t_mmd_np.mean(), 1)
std_DD_mmd = np.round(t_mmd_np.std(), 1)
print('M-statistic', DD_mmd, '±', std_DD_mmd)

t_cusum_np = np.array(t_cusum)
DD_cusum = np.round(t_cusum_np.mean(), 1)
std_DD_cusum = np.round(t_cusum_np.std(), 1)
print('CUSUM', DD_cusum, '±', std_DD_cusum)

KLIEP 8.9 ± 3.6
M-statistic 10.4 ± 3.4
CUSUM 5.0 ± 2.0


### Variance change detection

In [11]:
np.random.seed(1)

# Number of observations
n = 150
# True change point
tau = 75

# Standard deviation before the change point
sigma_1 = 0.1
# Standard deviation after the change point
sigma_2 = 0.3

p = 3
S_poly_list = []
S_nn_list = []

for item in range(10):
    
    print('Iteration', item)
    
    # Generate a Gaussian sequence of observations
    X = randn(n)
    X[:tau] *= sigma_1
    X[tau:] *= sigma_2

    S_poly, _ = compute_test_stat_linear(X, p)
    S_nn, _ = compute_test_stat_nn(X, n_epochs=50)
    
    S_poly_list.append(S_poly)
    S_nn_list.append(S_nn)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [12]:
t_lin = []
t_nn = []

fa_lin = 0
fa_nn = 0

for i in range(10):

    S_lin = S_poly_list[i]
    S_nn = S_nn_list[i]
    imin_lin, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_lin, S_lin <= threshold_p_3_poly))
    imin_nn, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_nn, S_nn <= threshold_nn))
    
    if imin_lin - tau <= 0:
        fa_lin += 1
    else:
        t_lin.append(imin_lin - tau)
        
    if imin_nn - tau <= 0:
        fa_nn += 1
    else:
        t_nn.append(imin_nn - tau)
        
print('Neural networks. False alarms:', fa_nn)
print('Polynomials. False alarms:', fa_lin)

Neural networks. False alarms: 0
Polynomials. False alarms: 0


In [13]:
t_lin_np = np.array(t_lin)
DD_poly = np.round(t_lin_np.mean(), 1)
std_DD_poly = np.round(t_lin_np.std(), 1)
print('polynomial basis', DD_poly, '±', std_DD_poly)

t_nn_np = np.array(t_nn)
DD_nn = np.round(t_nn_np.mean(), 1)
std_DD_nn = np.round(t_nn_np.std(), 1)
print('neural network', DD_nn, '±', std_DD_nn)

polynomial basis 16.4 ± 8.1
neural network 18.7 ± 9.2


In [14]:
np.random.seed(1)

# Number of observations
n = 150
# True change point
tau = 75

# Standard deviation before the change point
sigma_1 = 0.1
# Standard deviation after the change point
sigma_2 = 0.3

p = 3
S_hermite_list = []
S_legendre_list = []

for item in range(10):
    
    print('Iteration', item)
    
    # Generate a Gaussian sequence of observations
    X = randn(n)
    X[:tau] *= sigma_1
    X[tau:] *= sigma_2

    S_hermite, _ = compute_test_stat_ftal(X, p, beta=0.01, design='hermite')
    S_legendre, _ = compute_test_stat_ftal(X, p, beta=0.01, design='legendre')
    
    S_hermite_list.append(S_hermite)
    S_legendre_list.append(S_legendre)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [15]:
t_hermite = []
t_legendre = []

fa_hermite = 0
fa_legendre = 0

for i in range(10):

    S_hermite = S_hermite_list[i]
    S_legendre = S_legendre_list[i]
    imin_hermite, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_hermite,\
                                                                   S_hermite <= threshold_hermite_p_3_001))
    imin_legendre, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_legendre,\
                                                                    S_legendre <= threshold_legendre_p_3_001))
    
    if imin_hermite - tau <= 0:
        fa_hermite += 1
    else:
        t_hermite.append(imin_hermite - tau)
        
    if imin_legendre - tau <= 0:
        fa_legendre += 1
    else:
        t_legendre.append(imin_legendre - tau)
        
print('Hermite polynomials. False alarms:', fa_hermite)
print('Legendre polynomials. False alarms:', fa_legendre)

Hermite polynomials. False alarms: 0
Legendre polynomials. False alarms: 0


In [16]:
t_hermite_np = np.array(t_hermite)
DD_hermite = np.round(t_hermite_np.mean(), 1)
std_DD_hermite = np.round(t_hermite_np.std(), 1)
print('FTAL Hermite', DD_hermite, '±', std_DD_hermite)

t_legendre_np = np.array(t_legendre)
DD_legendre = np.round(t_legendre_np.mean(), 1)
std_DD_legendre = np.round(t_legendre_np.std(), 1)
print('FTAL Legendre', DD_legendre, '±', std_DD_legendre)

FTAL Hermite 14.1 ± 5.9
FTAL Legendre 14.6 ± 6.8


In [17]:
np.random.seed(1)

# Number of observations
n = 150
# True change point
tau = 75

# Standard deviation before the change point
sigma_1 = 0.1
# Standard deviation after the change point
sigma_2 = 0.3

S_kliep_list = []
S_mmd_list = []

for item in range(10):
    # Generate a Gaussian sequence of observations

    X = randn(n)
    X[:tau] *= sigma_1
    X[tau:] *= sigma_2
    
    S_kliep, _ = compute_test_stat_kliep(X, window_size=20, sigma=0.33)
    S_mmd, _ = compute_test_stat_mmd(X, window_size=20, sigma=0.1)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
    print(item)

0
1
2
3
4
5
6
7
8
9


In [18]:
t_kliep = []
t_mmd = []

fa_kliep = 0
fa_mmd = 0

for i in range(10):

    S_kliep = S_kliep_list[i]
    S_mmd = S_mmd_list[i]
    
    if np.sum(S_kliep > threshold_kliep_33) > 0:
        imin_kliep, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_kliep, S_kliep <= threshold_kliep_33))
    else:
        imin_kliep = n
    
    if np.sum(S_mmd > threshold_mmd_20_10) > 0:
        imin_mmd, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_mmd, S_mmd <= threshold_mmd_20_10))
    else:
        imin_mmd = n
        
    
    if imin_kliep - tau <= 0:
        fa_kliep += 1
    else:
        t_kliep.append(imin_kliep - tau)
        
    if imin_mmd - tau <= 0:
        fa_mmd += 1
    else:
        t_mmd.append(imin_mmd - tau)
        
print('KLIEP. False alarms:', fa_kliep)
print('M-statistic. False alarms:', fa_mmd)

KLIEP. False alarms: 0
M-statistic. False alarms: 0


In [19]:
t_kliep_np = np.array(t_kliep)
DD_kliep = np.round(t_kliep_np.mean(), 1)
std_DD_kliep = np.round(t_kliep_np.std(), 1)
print('KLIEP', DD_kliep, '±', std_DD_kliep)

t_mmd_np = np.array(t_mmd)
DD_mmd = np.round(t_mmd_np.mean(), 1)
std_DD_mmd = np.round(t_mmd_np.std(), 1)
print('M-statistic', DD_mmd, '±', std_DD_mmd)

KLIEP 19.2 ± 18.4
M-statistic 51.1 ± 27.3


### Distribution change detection

In [20]:
np.random.seed(1)
sigma = 0.1

# Number of observations
n = 200
# True change point
tau = 75

p = 6
S_poly_list = []
S_nn_list = []

for item in range(5):
    
    print('Iteration', item)
    
    ### Uniform on [-sigma sqrt(3), sigma sqrt(3)]
    Y = 2 * np.sqrt(3) * sigma * (np.random.rand(tau) - 0.5)
    ### Gaussian N(0, sigma^2)
    Z = sigma * randn(n - tau)
    X = np.append(Y, Z)
    
    S_poly, _ = compute_test_stat_linear(X, p)
    S_nn, _ = compute_test_stat_nn(X, n_epochs=50)
    
    S_poly_list.append(S_poly)
    S_nn_list.append(S_nn)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4


In [21]:
t_lin = []
t_nn = []

fa_lin = 0
fa_nn = 0

nd_nn = 0
nd_lin = 0

for i in range(5):

    S_lin = S_poly_list[i]
    S_nn = S_nn_list[i]
    
    if np.sum(S_lin > threshold_p_6_poly) > 0:
        imin_lin, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_lin, S_lin <= threshold_p_6_poly))
    else:
        imin_lin = n
        nd_lin += 1
        
    if np.sum(S_nn > threshold_nn_4) > 0:
        imin_nn, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_nn, S_nn <= threshold_nn_4))
    else:
        imin_nn = n
        nd_nn += 1
    
    if imin_lin - tau <= 0:
        fa_lin += 1
    else:
        t_lin.append(imin_lin - tau)
        
    if imin_nn - tau <= 0:
        fa_nn += 1
    else:
        t_nn.append(imin_nn - tau)
        
print('Neural networks. False alarms:', fa_nn, '. Undetected change points:', nd_nn)
print('Polynomials. False alarms:', fa_lin, '. Undetected change points:', nd_lin)

Neural networks. False alarms: 1 . Undetected change points: 0
Polynomials. False alarms: 0 . Undetected change points: 1


In [22]:
t_lin_np = np.array(t_lin)
DD_poly = np.round(t_lin_np.mean(), 1)
std_DD_poly = np.round(t_lin_np.std(), 1)
print('polynomial basis', DD_poly, '±', std_DD_poly)

t_nn_np = np.array(t_nn)
DD_nn = np.round(t_nn_np.mean(), 1)
std_DD_nn = np.round(t_nn_np.std(), 1)
print('neural network', DD_nn, '±', std_DD_nn)

polynomial basis 58.4 ± 34.8
neural network 38.0 ± 14.1


In [23]:
np.random.seed(1)

# Number of observations
n = 200
# True change point
tau = 75

sigma = 0.1

p = 6
S_hermite_list = []
S_legendre_list = []

for item in range(5):
    
    print('Iteration', item)
    
    ### Uniform on [-sigma sqrt(3), sigma sqrt(3)]
    Y = 2 * np.sqrt(3) * sigma * (np.random.rand(tau) - 0.5)
    ### Gaussian N(0, sigma^2)
    Z = sigma * randn(n - tau)
    X = np.append(Y, Z)
    
    S_hermite, _ = compute_test_stat_ftal(X, p, beta=0.005, design='hermite')
    S_legendre, _ = compute_test_stat_ftal(X, p, beta=0.005, design='legendre')
    
    S_hermite_list.append(S_hermite)
    S_legendre_list.append(S_legendre)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4


In [24]:
t_hermite = []
t_legendre = []

fa_hermite = 0
fa_legendre = 0

nd_hermite = 0
nd_legendre = 0

for i in range(5):

    S_hermite = S_hermite_list[i]
    S_legendre = S_legendre_list[i]
    
    if np.sum(S_hermite > threshold_hermite_p_6_0005) > 0:
        imin_hermite, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_hermite,\
                                                                       S_hermite <= threshold_hermite_p_6_0005))
    else:
        imin_hermite = n
        nd_hermite += 1
        
    if np.sum(S_legendre > threshold_legendre_p_6_0005) > 0:
        imin_legendre, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_legendre,\
                                                                        S_legendre <= threshold_legendre_p_6_0005))
    else:
        imin_legendre = n
        nd_legendre += 1
    
    if imin_hermite - tau <= 0:
        fa_hermite += 1
    else:
        t_hermite.append(imin_hermite - tau)
        
    if imin_legendre - tau <= 0:
        fa_legendre += 1
    else:
        t_legendre.append(imin_legendre - tau)
        
print('Hermite polynomials. False alarms:', fa_hermite, '. Undetected change points:', nd_hermite)
print('Legendre polynomials. False alarms:', fa_legendre, '. Undetected change points:', nd_legendre)

Hermite polynomials. False alarms: 0 . Undetected change points: 0
Legendre polynomials. False alarms: 0 . Undetected change points: 0


In [25]:
t_hermite_np = np.array(t_hermite)
DD_hermite = np.round(t_hermite_np.mean(), 1)
std_DD_hermite = np.round(t_hermite_np.std(), 1)
print('FTAL Hermite', DD_hermite, '±', std_DD_hermite)

t_legendre_np = np.array(t_legendre)
DD_legendre = np.round(t_legendre_np.mean(), 1)
std_DD_legendre = np.round(t_legendre_np.std(), 1)
print('FTAL Legendre', DD_legendre, '±', std_DD_legendre)

FTAL Hermite 24.2 ± 19.7
FTAL Legendre 17.6 ± 14.7


In [26]:
np.random.seed(1)

# Number of observations
n = 200
# True change point
tau = 75

sigma = 0.1

S_kliep_list = []
S_mmd_list = []

for item in range(5):
    
    ### Uniform on [-sigma sqrt(3), sigma sqrt(3)]
    Y = 2 * np.sqrt(3) * sigma * (np.random.rand(tau) - 0.5)
    ### Gaussian N(0, sigma^2)
    Z = sigma * randn(n - tau)
    X = np.append(Y, Z)
    
    S_kliep, _ = compute_test_stat_kliep(X, window_size=50, sigma=0.5)
    S_mmd, _ = compute_test_stat_mmd(X, window_size=50, sigma=0.5)
    
    S_kliep_list.append(S_kliep)
    S_mmd_list.append(S_mmd)
    
    print(item)

0
1
2
3
4


In [28]:
t_kliep = []
t_mmd = []

fa_kliep = 0
fa_mmd = 0

nd_kliep = 0
nd_mmd = 0

for i in range(5):

    S_kliep = S_kliep_list[i]
    S_mmd = S_mmd_list[i]
    
    if np.sum(S_kliep > threshold_kliep_50) > 0:
        imin_kliep, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_kliep, S_kliep <= threshold_kliep_50))
    else:
        imin_kliep = n
        nd_kliep += 1
    
    if np.sum(S_mmd > threshold_mmd_50_50) > 0:
        imin_mmd, _ = np.ma.flatnotmasked_edges(np.ma.masked_array(S_mmd, S_mmd <= threshold_mmd_50_50))
    else:
        imin_mmd = n
        nd_mmd += 1
        
    
    if imin_kliep - tau <= 0:
        fa_kliep += 1
    else:
        t_kliep.append(imin_kliep - tau)
        
    if imin_mmd - tau <= 0:
        fa_mmd += 1
    else:
        t_mmd.append(imin_mmd - tau)
        
print('KLIEP. False alarms:', fa_kliep, '. Undetected change points:', nd_kliep)
print('M-statistic. False alarms:', fa_mmd, '. Undetected change points:', nd_mmd)

KLIEP. False alarms: 0 . Undetected change points: 0
M-statistic. False alarms: 0 . Undetected change points: 0


In [29]:
t_kliep_np = np.array(t_kliep)
DD_kliep = np.round(t_kliep_np.mean(), 1)
std_DD_kliep = np.round(t_kliep_np.std(), 1)
print('KLIEP', DD_kliep, '±', std_DD_kliep)

t_mmd_np = np.array(t_mmd)
DD_mmd = np.round(t_mmd_np.mean(), 1)
std_DD_mmd = np.round(t_mmd_np.std(), 1)
print('M-statistic', DD_mmd, '±', std_DD_mmd)

KLIEP 26.8 ± 1.6
M-statistic 27.8 ± 2.2
